**Reddit Set Testing File**
7/08/2022

In [5]:
# dataloader
import pandas as pd

data_df = pd.read_csv('reddit_set.csv')

In [6]:
# train test split
from sklearn.model_selection import train_test_split

# training = 0.7, validation = 0.1, test = 0.2
training_tweets, temp_tweets, training_labels, temp_labels = train_test_split(list(data_df['Text_data']),list(data_df['Label']), test_size=0.3, random_state=123)
val_tweets, test_tweets, val_labels, test_labels = train_test_split(temp_tweets, temp_labels, test_size=(1/3), random_state=123)

In [7]:
# create dict
train = {'text': training_tweets, 'label': training_labels}
val = {'text': val_tweets, 'label': val_labels}
test = {'text': test_tweets, 'label': test_labels}

In [8]:
# convert to dataset
from datasets import Dataset

train_dataset = Dataset.from_dict(train)
val_dataset = Dataset.from_dict(val)
test_dataset = Dataset.from_dict(test)

In [9]:
# tokenize function
def tokenize_function(dataset):
    model_inputs = tokenizer(dataset['text'], padding="max_length", truncation=True, max_length=300)
    return model_inputs

In [10]:
def tokenize_set():
    tok_train_dataset = train_dataset.map(tokenize_function, batched=True)
    tok_val_dataset = val_dataset.map(tokenize_function, batched=True)
    tok_test_dataset = test_dataset.map(tokenize_function, batched=True)
    return tok_train_dataset,tok_val_dataset,tok_test_dataset

In [11]:
# training args
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="transformer_checkpoints",
    num_train_epochs=10
)

In [12]:
# train model
from transformers import Trainer

def train_transformer(model,tok_train_dataset, tok_val_dataset):

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tok_train_dataset,
        eval_dataset=tok_val_dataset,
    )

    trainer.train()
    return model

In [13]:
# predict test set
import numpy as np
import torch

def predict_nn(trained_model, tok_test_dataset):

    output = trained_model(attention_mask=torch.tensor(tok_test_dataset["attention_mask"]).cuda(), input_ids=torch.tensor(tok_test_dataset["input_ids"]).cuda())

    pred_labs = np.argmax(output["logits"].cpu().detach().numpy(), axis=1)

    gold_labs = tok_test_dataset["label"]

    return gold_labs, pred_labs

In [14]:
def stats(model):
    return predict_nn(model, test_dataset)

In [15]:
# empty gpu
import gc

def empty_gpu():
    model = None
    gc.collect()
    torch.cuda.empty_cache()

In [21]:
from transformers import BertTokenizer,RobertaTokenizer, ElectraForSequenceClassification, AutoModelForSequenceClassification, RobertaForSequenceClassification
from sklearn.metrics import accuracy_score, f1_score

model_name = []
acc_scores = []
f1_scores = []

tokenizer = BertTokenizer.from_pretrained('prajjwal1/bert-tiny')
train, val, test = tokenize_set()
model = AutoModelForSequenceClassification.from_pretrained('prajjwal1/bert-tiny', num_labels=3)
for param in model.bert.parameters():
    param.requires_grad = False
trained_model = train_transformer(model, train, val)
gold, pred = predict_nn(trained_model, test)
model_name += ['prajjwal1/bert-tiny']
acc_scores += [accuracy_score(gold,pred)]
f1_scores+= [f1_score(gold,pred, average='macro')]
empty_gpu()

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
train, val, test = tokenize_set()
model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)
for param in model.bert.parameters():
    param.requires_grad = False
trained_model = train_transformer(model, train, val)
gold, pred = predict_nn(trained_model, test)
model_name += ['bert-base-uncased']
acc_scores += [accuracy_score(gold,pred)]
f1_scores+= [f1_score(gold,pred, average='macro')]
empty_gpu()

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
train, val, test = tokenize_set()
model = AutoModelForSequenceClassification.from_pretrained('bert-base-cased', num_labels=3)
for param in model.bert.parameters():
    param.requires_grad = False
trained_model = train_transformer(model, train, val)
gold, pred = predict_nn(trained_model, test)
model_name += ['bert-base-cased']
acc_scores += [accuracy_score(gold,pred)]
f1_scores+= [f1_score(gold,pred, average='macro')]
empty_gpu()


tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
train, val, test = tokenize_set()
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=3)
for param in model.roberta.parameters():
    param.requires_grad = False
trained_model = train_transformer(model, train, val)
gold, pred = predict_nn(trained_model, test)
model_name += ['roberta-base']
acc_scores += [accuracy_score(gold,pred)]
f1_scores+= [f1_score(gold,pred, average='macro')]
empty_gpu()

tokenizer = RobertaTokenizer.from_pretrained('roberta-large')
train, val, test = tokenize_set()
model = RobertaForSequenceClassification.from_pretrained('roberta-large', num_labels=3)
for param in model.roberta.parameters():
    param.requires_grad = False
trained_model = train_transformer(model, train, val)
gold, pred = predict_nn(trained_model, test)
model_name += ['roberta-large']
acc_scores += [accuracy_score(gold,pred)]
f1_scores+= [f1_score(gold,pred, average='macro')]
empty_gpu()



loading file https://huggingface.co/prajjwal1/bert-tiny/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/585ac1c3dedc6b808dd35e8770afafe10905d3e723a02617af749d39db780e09.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
loading file https://huggingface.co/prajjwal1/bert-tiny/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/prajjwal1/bert-tiny/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/prajjwal1/bert-tiny/resolve/main/tokenizer_config.json from cache at None
loading configuration file https://huggingface.co/prajjwal1/bert-tiny/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3cf34679007e9fe5d0acd644dcc1f4b26bec5cbc9612364f6da7262aed4ef7a4.a5a11219cf90aae61ff30e1658ccf2cb4aa84d6b6e947336556f887c9828dc6d
Model config BertConfig {
  "_name_or_path": "prajjwal1/bert-tiny",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": nul

  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

loading configuration file https://huggingface.co/prajjwal1/bert-tiny/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3cf34679007e9fe5d0acd644dcc1f4b26bec5cbc9612364f6da7262aed4ef7a4.a5a11219cf90aae61ff30e1658ccf2cb4aa84d6b6e947336556f887c9828dc6d
Model config BertConfig {
  "_name_or_path": "prajjwal1/bert-tiny",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 128,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 512,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 2,
  "num_hidden_layers": 2,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.21.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 3

...

loading configuration file https://huggingface.co/roberta-large/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/dea67b44b38d504f2523f3ddb6acb601b23d67bee52c942da336fa1283100990.94cae8b3a8dbab1d59b9d4827f7ce79e73124efa6bb970412cd503383a95f373
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.21.1",
  "type

storing https://huggingface.co/roberta-large/resolve/main/pytorch_model.bin in cache at /root/.cache/huggingface/transformers/8e36ec2f5052bec1e79e139b84c2c3089cb647694ba0f4f634fec7b8258f7c89.c43841d8c5cd23c435408295164cda9525270aa42cd0cc9200911570c0342352
creating metadata file for /root/.cache/huggingface/transformers/8e36ec2f5052bec1e79e139b84c2c3089cb647694ba0f4f634fec7b8258f7c89.c43841d8c5cd23c435408295164cda9525270aa42cd0cc9200911570c0342352
loading weights file https://huggingface.co/roberta-large/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/8e36ec2f5052bec1e79e139b84c2c3089cb647694ba0f4f634fec7b8258f7c89.c43841d8c5cd23c435408295164cda9525270aa42cd0cc9200911570c0342352
Some weights of the model checkpoint at roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'roberta.pool

Step,Training Loss
500,0.934100
1000,0.898200
1500,0.878600


Saving model checkpoint to transformer_checkpoints/checkpoint-500
Configuration saved in transformer_checkpoints/checkpoint-500/config.json
Model weights saved in transformer_checkpoints/checkpoint-500/pytorch_model.bin
Saving model checkpoint to transformer_checkpoints/checkpoint-1000
Configuration saved in transformer_checkpoints/checkpoint-1000/config.json
Model weights saved in transformer_checkpoints/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to transformer_checkpoints/checkpoint-1500
Configuration saved in transformer_checkpoints/checkpoint-1500/config.json
Model weights saved in transformer_checkpoints/checkpoint-1500/pytorch_model.bin


KeyboardInterrupt: ignored

In [22]:
tokenizer = BertTokenizer.from_pretrained('google/electra-small-discriminator')
train, val, test = tokenize_set()
model = ElectraForSequenceClassification.from_pretrained('google/electra-small-discriminator', num_labels=3)
for param in model.electra.parameters():
    param.requires_grad = False
trained_model = train_transformer(model, train, val)
gold, pred = predict_nn(trained_model, test)
model_name += ['google/electra-small-discriminator']
acc_scores += [accuracy_score(gold,pred)]
f1_scores+= [f1_score(gold,pred, average='macro')]
empty_gpu()

tokenizer = BertTokenizer.from_pretrained('google/electra-base-discriminator')
train, val, test = tokenize_set()
model = ElectraForSequenceClassification.from_pretrained('google/electra-base-discriminator', num_labels=3)
for param in model.electra.parameters():
    param.requires_grad = False
trained_model = train_transformer(model, train, val)
gold, pred = predict_nn(trained_model, test)
model_name += ['google/electra-base-discriminator']
acc_scores += [accuracy_score(gold,pred)]
f1_scores += [f1_score(gold,pred, average='macro')]
empty_gpu()

tokenizer = BertTokenizer.from_pretrained('google/electra-base-generator')
train, val, test = tokenize_set()
model = ElectraForSequenceClassification.from_pretrained('google/electra-base-generator', num_labels=3)
for param in model.electra.parameters():
    param.requires_grad = False
trained_model = train_transformer(model, train, val)
gold, pred = predict_nn(trained_model, test)
model_name += ['google/electra-base-generator']
acc_scores += [accuracy_score(gold,pred)]
f1_scores+= [f1_score(gold,pred, average='macro')]
empty_gpu()

https://huggingface.co/google/electra-small-discriminator/resolve/main/vocab.txt not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpdh6efpjq


storing https://huggingface.co/google/electra-small-discriminator/resolve/main/vocab.txt in cache at /root/.cache/huggingface/transformers/ece45ade3e01224cf31fed8e183b306d17b84e8abd415363474cfe72274f7814.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
creating metadata file for /root/.cache/huggingface/transformers/ece45ade3e01224cf31fed8e183b306d17b84e8abd415363474cfe72274f7814.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
https://huggingface.co/google/electra-small-discriminator/resolve/main/tokenizer_config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmp_d5av5zl


storing https://huggingface.co/google/electra-small-discriminator/resolve/main/tokenizer_config.json in cache at /root/.cache/huggingface/transformers/8b3aea9f7242b3d19268df5b1bfed8f66e08671a72ac0809ada08e5ef1adc592.19eda9a6da5fb0e52a45200c95876729561dde16a69b9116953af6edca1d1e92
creating metadata file for /root/.cache/huggingface/transformers/8b3aea9f7242b3d19268df5b1bfed8f66e08671a72ac0809ada08e5ef1adc592.19eda9a6da5fb0e52a45200c95876729561dde16a69b9116953af6edca1d1e92
loading file https://huggingface.co/google/electra-small-discriminator/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/ece45ade3e01224cf31fed8e183b306d17b84e8abd415363474cfe72274f7814.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
loading file https://huggingface.co/google/electra-small-discriminator/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/google/electra-small-discriminator/resolve/main/special_tokens_map.json from cache at None


storing https://huggingface.co/google/electra-small-discriminator/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/ca13c16218c6780ec76753d3afa19fcb7cc759e3f63ee87e441562d374762b3d.3dd1921e571dfa18c0bdaa17b9b38f111097812281989b1cb22263738e66ef73
creating metadata file for /root/.cache/huggingface/transformers/ca13c16218c6780ec76753d3afa19fcb7cc759e3f63ee87e441562d374762b3d.3dd1921e571dfa18c0bdaa17b9b38f111097812281989b1cb22263738e66ef73
loading configuration file https://huggingface.co/google/electra-small-discriminator/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/ca13c16218c6780ec76753d3afa19fcb7cc759e3f63ee87e441562d374762b3d.3dd1921e571dfa18c0bdaa17b9b38f111097812281989b1cb22263738e66ef73
Model config ElectraConfig {
  "_name_or_path": "google/electra-small-discriminator",
  "architectures": [
    "ElectraForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "embedding_size": 128,
  "hidde

  0%|          | 0/5 [00:00<?, ?ba/s]

...

  0%|          | 0/1 [00:00<?, ?ba/s]

loading configuration file https://huggingface.co/google/electra-base-generator/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/1b8851729292ac310aa08b6d130a27579aebd8557635c997b1940825c1b543b9.8cfd0b697dea70b75d1b291b655114b572235f235c65599c6e82ab6b858f69d3
Model config ElectraConfig {
  "architectures": [
    "ElectraForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "embedding_size": 768,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 256,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "num_attention_heads": 4,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "summary_activation": "gelu",
  "summary_las

storing https://huggingface.co/google/electra-base-generator/resolve/main/pytorch_model.bin in cache at /root/.cache/huggingface/transformers/aa8b1a6defad3704fa06d906422dba45d9d2fc7f6a0a10e4f6d6e6ea573e9551.2a44944123c60252346c02589e5f73650d732d44a7335fdeb61f189131fd17ac
creating metadata file for /root/.cache/huggingface/transformers/aa8b1a6defad3704fa06d906422dba45d9d2fc7f6a0a10e4f6d6e6ea573e9551.2a44944123c60252346c02589e5f73650d732d44a7335fdeb61f189131fd17ac
loading weights file https://huggingface.co/google/electra-base-generator/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/aa8b1a6defad3704fa06d906422dba45d9d2fc7f6a0a10e4f6d6e6ea573e9551.2a44944123c60252346c02589e5f73650d732d44a7335fdeb61f189131fd17ac
Some weights of the model checkpoint at google/electra-base-generator were not used when initializing ElectraForSequenceClassification: ['generator_predictions.LayerNorm.bias', 'generator_predictions.dense.weight', 'generator_predictions.LayerNor

Step,Training Loss
500,0.982800
1000,0.948400
1500,0.943100
2000,0.930000
2500,0.936800
3000,0.931200
3500,0.918500
4000,0.919400
4500,0.921100
5000,0.914800


Saving model checkpoint to transformer_checkpoints/checkpoint-500
Configuration saved in transformer_checkpoints/checkpoint-500/config.json
Model weights saved in transformer_checkpoints/checkpoint-500/pytorch_model.bin
Saving model checkpoint to transformer_checkpoints/checkpoint-1000
Configuration saved in transformer_checkpoints/checkpoint-1000/config.json
Model weights saved in transformer_checkpoints/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to transformer_checkpoints/checkpoint-1500
Configuration saved in transformer_checkpoints/checkpoint-1500/config.json
Model weights saved in transformer_checkpoints/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to transformer_checkpoints/checkpoint-2000
Configuration saved in transformer_checkpoints/checkpoint-2000/config.json
Model weights saved in transformer_checkpoints/checkpoint-2000/pytorch_model.bin
Saving model checkpoint to transformer_checkpoints/checkpoint-2500
Configuration saved in transformer_checkpoin

In [24]:
data = {'Model': model_name, 'Accuracy': acc_scores, 'Macro F1': f1_scores}
df = pd.DataFrame(data)
df.to_csv('reddit_results_frozen')

In [25]:
df

,Model,Accuracy,Macro F1
0,prajjwal1/bert-tiny,0.561224,0.393064
1,bert-base-uncased,0.562925,0.396493
2,bert-base-cased,0.528912,0.372102
3,roberta-base,0.588435,0.415452
4,google/electra-small-discriminator,0.562925,0.394492
5,google/electra-base-discriminator,0.573129,0.402446
6,google/electra-base-generator,0.540816,0.378078
